In [49]:
import datetime as dt
import time
#import matplotlib
import matplotlib.pyplot as plt
import numpy as np
#import scipy as sp

import pyxu.abc as pxa
import pyxu.operator as pxop
import pyxu.opt.stop as pxos
#from pyxu.opt.solver.pgd import PGD

import pfw_hawkes as pyfwl
from hawkes_likelihood import HawkesLikelihood
import utils as ut

seed = 1  # None  # for reproducibility
if seed is None:
    seed = np.random.randint(0, 1000)
print(f"Seed: {seed}")

Seed: 1


In [3]:
%load_ext autoreload
%autoreload 2

%matplotlib qt

### Import data

In [48]:
path = "turtle_data/events0.csv"
beta = 5
hpL = HawkesLikelihood(path=path, beta=beta)

Computing A^1 out of 200, k1=75
Computing A^2 out of 200, k2=288
Computing A^3 out of 200, k3=694
Computing A^4 out of 200, k4=29
Computing A^5 out of 200, k5=176
Computing A^6 out of 200, k6=209
Computing A^7 out of 200, k7=83
Computing A^8 out of 200, k8=301
Computing A^9 out of 200, k9=1727
Computing A^10 out of 200, k10=556
Computing A^11 out of 200, k11=141
Computing A^12 out of 200, k12=1114
Computing A^13 out of 200, k13=342
Computing A^14 out of 200, k14=390
Computing A^15 out of 200, k15=499
Computing A^16 out of 200, k16=616
Computing A^17 out of 200, k17=543
Computing A^18 out of 200, k18=386
Computing A^19 out of 200, k19=710
Computing A^20 out of 200, k20=210
Computing A^21 out of 200, k21=218
Computing A^22 out of 200, k22=211
Computing A^23 out of 200, k23=485
Computing A^24 out of 200, k24=321
Computing A^25 out of 200, k25=305
Computing A^26 out of 200, k26=1088
Computing A^27 out of 200, k27=678
Computing A^28 out of 200, k28=467
Computing A^29 out of 200, k29=140
Com

In [79]:
hpL1 = HawkesLikelihood(path="turtle_data/events1.csv", beta=beta)

Computing A^1 out of 64, k1=293
Computing A^2 out of 64, k2=1221
Computing A^3 out of 64, k3=222
Computing A^4 out of 64, k4=16
Computing A^5 out of 64, k5=9
Computing A^6 out of 64, k6=10
Computing A^7 out of 64, k7=749
Computing A^8 out of 64, k8=705
Computing A^9 out of 64, k9=784
Computing A^10 out of 64, k10=566
Computing A^11 out of 64, k11=6176
Computing A^12 out of 64, k12=1625
Computing A^13 out of 64, k13=3741
Computing A^14 out of 64, k14=1463
Computing A^15 out of 64, k15=10815
Computing A^16 out of 64, k16=934
Computing A^17 out of 64, k17=5105
Computing A^18 out of 64, k18=3340
Computing A^19 out of 64, k19=644
Computing A^20 out of 64, k20=317
Computing A^21 out of 64, k21=5061
Computing A^22 out of 64, k22=81
Computing A^23 out of 64, k23=8
Computing A^24 out of 64, k24=2925
Computing A^25 out of 64, k25=890
Computing A^26 out of 64, k26=11
Computing A^27 out of 64, k27=149
Computing A^28 out of 64, k28=668
Computing A^29 out of 64, k29=4514
Computing A^30 out of 64, k3

In [92]:
hpL2 = HawkesLikelihood(path="turtle_data/events2.csv", beta=beta)

Computing A^1 out of 214, k1=86
Computing A^2 out of 214, k2=149
Computing A^3 out of 214, k3=298
Computing A^4 out of 214, k4=4433
Computing A^5 out of 214, k5=251
Computing A^6 out of 214, k6=157
Computing A^7 out of 214, k7=20
Computing A^8 out of 214, k8=285
Computing A^9 out of 214, k9=205
Computing A^10 out of 214, k10=1505
Computing A^11 out of 214, k11=662
Computing A^12 out of 214, k12=74
Computing A^13 out of 214, k13=378
Computing A^14 out of 214, k14=283
Computing A^15 out of 214, k15=102
Computing A^16 out of 214, k16=118
Computing A^17 out of 214, k17=512
Computing A^18 out of 214, k18=220
Computing A^19 out of 214, k19=155
Computing A^20 out of 214, k20=1222
Computing A^21 out of 214, k21=2578
Computing A^22 out of 214, k22=1037
Computing A^23 out of 214, k23=1709
Computing A^24 out of 214, k24=475
Computing A^25 out of 214, k25=583
Computing A^26 out of 214, k26=468
Computing A^27 out of 214, k27=1035
Computing A^28 out of 214, k28=37
Computing A^29 out of 214, k29=379


In [93]:
start = time.time()
lip = hpL2.opA.estimate_lipschitz(method='svd')
print("Computation time of the Lipschitz constant: {:.4f}".format(time.time()-start))
print("Lipschitz constant:", lip)
#np.linalg.svd to compute the largest singular value which is = to the lipschitz constant for linear operators

Computation time of the Lipschitz constant: 25.8925
Lipschitz constant: 978.3462949963274


In [94]:
hpL2.plot_realization()

### Define solver

In [95]:
# Parameter for reconstruction
#lambda_factor = 0.1

## Parameters of the solvers
# Base
remove = True
min_iterations = 1
# Stop
eps = 1e-4
tmax = 600.0
eps_dcv = 1e-2
# PFW
ms_threshold = 0.7  # original value 0.8, the lower the more atoms will HP select
init_correction = 1e-1
final_correction = 1e-6
correction_steps = 5

#lambda_ = lambda_factor * np.linalg.norm(op.T(measurements), np.infty)  # rule of thumb to define lambda
lambda_ = 1e1

In [96]:
pfw = pyfwl.PFWLasso(
    forwardOp=hpL2.opA,
    convexOp=hpL2.E,
    lambda_=lambda_,
    M=hpL2.M,
    negLogL=hpL2.negLogL,
    ms_threshold=ms_threshold,
    init_correction_prec=init_correction,
    final_correction_prec=final_correction,
    remove_positions=remove,
    min_correction_steps=correction_steps,
    show_progress=False,
)

In [97]:
stop_crit = pxos.RelError(
    eps=eps,
    var="objective_func",
    f=None,
    norm=2,
    satisfy_all=True,
)
# alternative stopping criteria
dcv = pyfwl.dcvStoppingCrit(eps_dcv)

# Minimum number of iterations
min_iter = pxos.MaxIter(n=min_iterations)

# Maximum duration
max_duration = pxos.MaxDuration(t=dt.timedelta(seconds=tmax))

stop = (min_iter & stop_crit) | max_duration

# track DCV
track_dcv = pxos.AbsError(eps=1e-10, var="dcv", f=None, norm=2, satisfy_all=True)

### Solving

In [98]:
print("Polyatomic FW: Solving ...")
start = time.time()
pfw.fit(stop_crit= stop | track_dcv, diff_lipschitz=lip**2)
data_p, hist_p = pfw.stats()
time_p = time.time() - start
print("\tSolved in {:.3f} seconds".format(time_p))


Polyatomic FW: Solving ...
diff_lipschitz constant provided.
initial value: [-3233.06622152]
new indices [  649   676   816  6454  6481  6621 36554 36581 36721 42769]
actual new indices: 10
new indices [  649   676   816  1945  6454  6481  6621  9874 36554 36581 36721 42574
 42601 42769 42783 45584 45611 45751 45779 45793]
actual new indices: 10
new indices [  649   676   816  1945  4095  4331  4471  6454  6481  6621  9874  9921
 19811 21886 24305 36554 36581 36721 42601 42769 42783 45584 45611 45751
 45793]
actual new indices: 7
new indices [  649   676   816  1945  4331  4471  6454  6481  6621  9874  9921 16969
 19811 19951 21886 25380 25390 25489 25569 36366 36506 36554 36581 36721
 42769 45751]
actual new indices: 8
new indices [  649   676   816  1945  4331  4761  6454  6481  6621  9874  9921 21886
 36581 36721]
actual new indices: 1
new indices [  649   676   816  1945  6454  6481  6621  7341  7481  9874  9921 21886
 36581 36721 36964 38684 44059 44073]
actual new indices: 6
new 

### Evaluation of the solve

In [99]:
print("Final value of dual certificate:\n\tPFW: {:.4f}".format(data_p["dcv"]))
print("Final value of objective function:\n\tPFW : {:.2f}".format(hist_p[-1][-1]))

Final value of dual certificate:
	PFW: 52.5247
Final value of objective function:
	PFW : -1388157.30


In [103]:
def supp(arr):
    return np.where(np.abs(arr) > 1e-3)[0]
M = hpL2.M

#print(data_p["x"][supp(data_p["x"])])
#print(data_p["x"])
# for i in range(M):
#     print(f"Process {i}:", data_p["x"][(i*(M + 1)):(i*(M + 1) + M + 1)])

# Arrange intensities in a M x M matrix
intensities = np.zeros((M, M))
for i in range(M):
    intensities[i, :] = data_p["x"][(i*(M + 1) + 1):(i*(M + 1) + M + 1)]

In [104]:
# Visualize solution
fig, ax = plt.subplots(1, 1, figsize=(5,5))
im = ax.imshow(intensities/(beta*beta), # la norme L1 est alpha/beta^2
                cmap='viridis', 
                interpolation='nearest', 
                extent=[1, M, M, 1])
fig.colorbar(im, ax=ax)  # Add a colorbar to show the scale
ax.set_title('L1 norms of the $h_{ij}$')
plt.show()

for i in range(M):
    print(f"mu{i+1}:", data_p["x"][i*(M + 1)])

print(intensities/(beta*beta))

mu1: 5.993774883997696
mu2: 6.114741024675324
mu3: 6.368580795358068
mu4: 8.102901100575336
mu5: 6.386873085344695
mu6: 6.176425353683839
mu7: 5.818803842810865
mu8: 6.348156250866688
mu9: 6.272743397614633
mu10: 7.323184545042934
mu11: 6.7144256371283495
mu12: 5.971653055771037
mu13: 6.622498326279062
mu14: 6.415764830449542
mu15: 6.017380280697673
mu16: 6.079008084184577
mu17: 6.586948714135114
mu18: 6.344138831376622
mu19: 6.124187604664219
mu20: 6.78395071533199
mu21: 8.014629943483975
mu22: 7.344229022289927
mu23: 7.83710389560777
mu24: 6.342024368759349
mu25: 7.0255189104436875
mu26: 6.868282984165228
mu27: 6.950048910686681
mu28: 5.872099443752778
mu29: 6.726579664459671
mu30: 5.9547285905384335
mu31: 8.168044775278789
mu32: 6.105495321907612
mu33: 6.2118792702220045
mu34: 7.3893879760838175
mu35: 7.373548912465734
mu36: 7.740668430869857
mu37: 6.957164707801368
mu38: 5.93060857120297
mu39: 7.525445537421062
mu40: 6.333658346836722
mu41: 5.942644695219119
mu42: 5.850938905107319

In [91]:
hpL1.plot_realization()